In [42]:
import cv2
import numpy as np
import matplotlib as plt
import random

filename = 'frog.png'
img = cv2.imread(filename)

img_confirm = img.copy()
img_confirm[img_confirm<250] = 0

img_display = np.zeros(img.shape,dtype=np.uint8)
img_display.fill(255)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

def isSegment(p1,p2, image):
    x1, y1 = p1
    x2, y2 = p2

    width, height, extra = image.shape
    
    line = 0
    notline = 0
    
    sample = []
    
    vote = 0
    for mult in range(10):  
        
        centx = x2 + random.randrange(0,int(max(1,x1-x2)))
        centy = y2 + random.randrange(0,int(max(1,y1-y2)))
        
        cl = 2 #radius of cluster points
        
        sample = image[int(centy-cl):int(centy+cl),int(centx-cl):int(centx+cl),:]
        w, h, i = sample.shape
        
        print sample
        if np.sum(sample) < 4000:
            vote+=1
    
    return vote>9

gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

ret, dst = cv2.threshold(dst,0.1*dst.max(),255,0)
dst = np.uint8(dst)
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

for i in range(1, len(corners)):
    print(corners[i])
#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.1*dst.max()]=[0,0,255]

found = 0
segments = dict()
counter = 0

for c1 in corners:
    for c2 in corners:
        if not ( set(c1) in segments.values() and set(c2) in segments) or (set(c2) in segments.values() and set(c1) in segments):
            if isSegment(c1, c2, img):
                segments.update({tuple(c1):tuple(c2)})

print "DETECTED SEGMENTS"
for i in segments:
    x1, y1 = i
    x2, y2 = segments.get(i)
    cv2.line(img_display,(x1,y1), (x2,y2), (0,0,255),1) 

    print "("+ str(i) +","+str(segments.get(i))+")"

cv2.circle(img_display, (int(1.6140463), int(199.0812)), 2, (0,255,0), thickness=1, lineType=8, shift=0) 
cv2.circle(img_display, (int(196.46725), int(197.349)), 2, (0,255,0), thickness=1, lineType=8, shift=0) 

test = isSegment((1.6140463, 199.0812),(196.46725, 197.349 ), img)
print test

cv2.imwrite("cornersdisplay.png", img_display)
cv2.imwrite("frogcorners.png", img)


[199.09956     1.6209096]
[2.6107898 2.5548253]
[395.45193     2.6043222]
[2.6102614 2.555428 ]
[395.45193     2.6043222]
[141.77531   57.471977]
[199.04112  57.11386]
[256.2257    57.471745]
[ 57.471977 141.77531 ]
[340.52795 141.77509]
[  1.6140463 199.08125  ]
[196.46725 197.349  ]
[200.65218 196.4671 ]
[396.37903 199.09973]
[ 57.11386 199.04112]
[340.88794 199.03578]
[196.4673  200.65193]
[200.65167 201.53317]
[ 57.471657 256.22568 ]
[340.5281 256.2259]
[141.7753  340.52805]
[199.03578 340.88794]
[256.22516 340.52786]
[  2.5399313 395.40097  ]
[395.3957 395.4519]
[  2.5398853 395.40094  ]
[199.07164 396.39062]
[395.39566 395.45197]
[[[  0   0 255]
  [  0   0 255]
  [ 67  67  67]
  [ 47  47  47]]

 [[  0   0 255]
  [ 11  11  11]
  [ 27  27  27]
  [ 12  12  12]]

 [[ 67  67  67]
  [ 27  27  27]
  [  3   3   3]
  [  0   0   0]]

 [[ 47  47  47]
  [ 12  12  12]
  [  0   0   0]
  [  0   0   0]]]
[[[  0   0 255]
  [  0   0 255]
  [ 67  67  67]
  [ 47  47  47]]

 [[  0   0 255]
  [ 11  11

[[[  2   2   2]
  [ 71  71  71]
  [ 91  91  91]
  [201 201 201]]

 [[  2   2   2]
  [ 66  66  66]
  [ 55  55  55]
  [185 185 185]]

 [[  2   2   2]
  [ 40  40  40]
  [ 94  94  94]
  [ 11  11  11]]

 [[  2   2   2]
  [ 10  10  10]
  [  6   6   6]
  [ 94  94  94]]]
[[[  2   2   2]
  [ 71  71  71]
  [ 91  91  91]
  [201 201 201]]

 [[  2   2   2]
  [ 66  66  66]
  [ 55  55  55]
  [185 185 185]]

 [[  2   2   2]
  [ 40  40  40]
  [ 94  94  94]
  [ 11  11  11]]

 [[  2   2   2]
  [ 10  10  10]
  [  6   6   6]
  [ 94  94  94]]]
[[[  2   2   2]
  [ 71  71  71]
  [ 91  91  91]
  [201 201 201]]

 [[  2   2   2]
  [ 66  66  66]
  [ 55  55  55]
  [185 185 185]]

 [[  2   2   2]
  [ 40  40  40]
  [ 94  94  94]
  [ 11  11  11]]

 [[  2   2   2]
  [ 10  10  10]
  [  6   6   6]
  [ 94  94  94]]]
[[[  2   2   2]
  [ 71  71  71]
  [ 91  91  91]
  [201 201 201]]

 [[  2   2   2]
  [ 66  66  66]
  [ 55  55  55]
  [185 185 185]]

 [[  2   2   2]
  [ 40  40  40]
  [ 94  94  94]
  [ 11  11  11]]

 [[  2   2

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[231 231 231]
  [231 231 231]
  [231 231 231]
  [231 231 231]]

 [[ 71  71  71]
  [ 71  71  71]
  [ 71  71  71]
  [ 71  71  71]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255

[[[120 120 120]
  [105 105 105]
  [254 254 254]
  [255 255 255]]

 [[255 255 255]
  [120 120 120]
  [105 105 105]
  [254 254 254]]

 [[255 255 255]
  [255 255 255]
  [120 120 120]
  [105 105 105]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [120 120 120]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255

[[[ 47  47  47]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[231 231 231]
  [231 231 231]
  [231 231 231]
  [231 231 231]]

 [[ 71  71

...

  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255

  [  0   0 255]]]
[[[169 169 169]
  [118 118 118]
  [255 255 255]
  [255 255 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255

  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [200 200 200]]

 [[240 240 240]
  [149 149 149]
  [ 53  53  53]
  [ 87  87  87]]

 [[ 48  48  48]
  [139 139 139]
  [234 234 234]
  [255 255 255]]

 [[254 254 254]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  [255 255

True

In [61]:
def isSegment(p1,p2):
    x1, y1 = p1
    x2, y2 = p2
    
    xsplit = (x1-x2)/3
    ysplit = (y1-y2)/3
    
    sample = np.empty(2)
    numpy.append(sample, (x1+xsplit,y1+ysplit))
    numpy.append(sample,(x1+2*xsplit,y1+2*ysplit))
    
    print sample
    
    
    
    